# Projet 6 :  Améliorez le produit IA de votre start-up
<font color = "cyan"><i>  Récupération des données et enregistrement sur fichier csv</i></font>

<font color = "grey"><i> Chargement des modules necessaires </i></font>

In [1]:
import os
import requests
import uuid
import json
import pandas as pd

## Sommaire
<h1><a href='#data_fetch'>I)Récupération des données</a></h1>
    <h2><a href='#data_fetch_api'>I.1) Utilisation de l'API Yelp</a></h2>
    <h2><a href='#data_save'>I.2) Enregistrement des données dans un dictionnaire</a></h2>
    <h2><a href='#data_gen'>I.3) Conversion en fichier csv</a></h2>

<a id='data_fetch'></a>
## I) Récupération des données

<a id='data_fetch_api'></a>
### I.1)  Utilisation de l'API
<font color = "cyan"><i>  requêtes **GET** pour obtenir les commentaires et les photos de 200 restaurants de Paris.</i></font>

In [2]:

env_key_yelp     = 'YELP_KEY'
env_endpoint_yelp = 'YELP_ENDPOINT'

if not env_key_yelp  in os.environ:
    print("No environment variable found for translator key.\n Enter key:")
    os.environ["YELP_KEY"] = input()
env_key_yelp = os.environ[env_key_yelp]

if not env_endpoint_yelp in os.environ:
    print("No environment variable found for endpoint translator.\n Enter key:")
    os.environ["YELP_ENDPOINT"] = input()

endpoint = os.environ[env_endpoint_yelp]
path = '/businesses/search'
constructed_url = endpoint + path
print("key :x",env_key_yelp)
headers = {'Authorization': 'Bearer %s' % env_key_yelp}

params = {'term':'restaurants',
          'location':'Paris',
          'offset':0, 'limit':50}

request = requests.get(constructed_url, params=params,
                       headers=headers)
print(constructed_url)
print('The status code is {}'.format(request.status_code))

response = request.text
print(response)
businesses = json.loads(response)['businesses']

key :x D47uwli50NFj4QTtEV4lZiYpGEgm7-GdQo49r0DV7OhiCQyZAXtuPMPQVyZO1eFixU5QKJwsrhpghei0zXU9IceeN2ZSxlzgj1b3yluznBAs6z1wmWZiRA4DcVFZYXYx
https://api.yelp.com/v3/businesses/search
The status code is 200
{"businesses": [{"id": "-0iLH7iQNYtoURciDpJf6w", "alias": "le-comptoir-de-la-gastronomie-paris", "name": "Le Comptoir de la Gastronomie", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/Je6THJBI0U0b1N47INfaKQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/le-comptoir-de-la-gastronomie-paris?adjust_creative=kbKyAG8-dQFg3Zw0SZOBMQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=kbKyAG8-dQFg3Zw0SZOBMQ", "review_count": 1107, "categories": [{"alias": "french", "title": "French"}], "rating": 4.5, "coordinates": {"latitude": 48.8645157999652, "longitude": 2.34540185646608}, "transactions": [], "price": "\u20ac\u20ac", "location": {"address1": "34 rue Montmartre", "address2": "", "address3": "", "city": "Paris", "zip_code": "75001", "country": "FR", "sta

<a id='data_save'></a>
### I.2) Enregistrement des données dans un dictionnaire

In [3]:
data = {'id': [], 'name': [], 'text_review': [], 'photo_1': [],
        'photo_2': [], 'photo_3': []}

data_1 = {'id': [], 'name': [], 'text_review': [], 'rating': []}

for elt in businesses:
    url = 'https://api.yelp.com/v3/businesses/' + elt['id'] + '/reviews'
    params = {'locale':'fr_FR'}
    req = requests.get(url, headers=headers, params=params)
    if req.status_code == 200:
        response = req.text
        reviews = json.loads(response)['reviews']
        text_reviews   = ""
        text_reviews_l = ""
        for elt_r in reviews:
            text_reviews = text_reviews + " " + elt_r['text']

        text_reviews_l = text_reviews
        url_1 = 'https://api.yelp.com/v3/businesses/' + elt['id']
        req_1 = requests.get(url_1, headers=headers)
        if req_1.status_code == 200:
            response_1 = req_1.text
            names = json.loads(response_1)['name']
            photos = json.loads(response_1)['photos']
            if (len(photos) == 3 and len(text_reviews_l) != 0):
                data['id'].append(elt['id'])
                data['name'].append(names)
                data['text_review'].append(text_reviews_l)
                data['photo_1'].append(photos[0])
                data['photo_2'].append(photos[1])
                data['photo_3'].append(photos[2])
                for elt_r in reviews:
                    data_1['id'].append(elt['id'])
                    data_1['name'].append(names)
                    data_1['text_review'].append(elt_r['text'])
                    data_1['rating'].append(elt_r['rating'])

<font color = "cyan"><i>  Récupération de 200 restaurants avec des avis négatifs</i></font>

In [4]:
i = 0
while len(data['id']) < 180:
    i += 50
    params = {'term':'restaurants', 'location':'Paris',
                  'offset':i, 'limit':50}

    request = requests.get(constructed_url, params=params,
                               headers=headers)
    if request.status_code == 200:

        response = request.text
        businesses = json.loads(response)['businesses']


        for elt in businesses:
            if  len(data['id']) < 180 :

                url = 'https://api.yelp.com/v3/businesses/' + elt['id'] + '/reviews'
                params = {'locale':'fr_FR'}
                req = requests.get(url, headers=headers, params=params)
                if req.status_code == 200:

                    response = req.text
                    reviews = json.loads(response)['reviews']
                    text_reviews   = ""
                    text_reviews_l = ""
                    for elt_r in reviews:
                        text_reviews = text_reviews + " " + elt_r['text']

                    text_reviews_l = text_reviews
                    url_1 = 'https://api.yelp.com/v3/businesses/' + elt['id']
                    req_1 = requests.get(url_1, headers=headers)
                    if req_1.status_code == 200:
                        response_1 = req_1.text
                        names = json.loads(response_1)['name']
                        photos = json.loads(response_1)['photos']
                        if (len(photos) == 3 and len(text_reviews_l) != 0)  :

                            data['id'].append(elt['id'])
                            data['name'].append(names)
                            data['text_review'].append(text_reviews_l)
                            data['photo_1'].append(photos[0])
                            data['photo_2'].append(photos[1])
                            data['photo_3'].append(photos[2])
                            for elt_r in reviews:
                                data_1['id'].append(elt['id'])
                                data_1['name'].append(names)
                                data_1['text_review'].append(elt_r['text'])
                                data_1['rating'].append(elt_r['rating'])
            else:
                elt=businesses


<a id='data_gen'></a>
### I.3) Conversion en fichier csv

In [5]:
df_yelp = pd.DataFrame.from_dict(data)
df_rating = pd.DataFrame.from_dict(data_1)


df_yelp.head()

,id,name,text_review,photo_1,photo_2,photo_3
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,J'y suis allé pour acheter un foie gras cru à...,https://s3-media2.fl.yelpcdn.com/bphoto/Je6THJ...,https://s3-media2.fl.yelpcdn.com/bphoto/Y0D70M...,https://s3-media4.fl.yelpcdn.com/bphoto/HetBW7...
1,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Les plats étaient très bons et suffisamment c...,https://s3-media1.fl.yelpcdn.com/bphoto/hPCZTb...,https://s3-media4.fl.yelpcdn.com/bphoto/Y5fZV7...,https://s3-media1.fl.yelpcdn.com/bphoto/lbhWbW...
2,cEjF41ZQB8-SST8cd3EsEw,L'Avant Comptoir,Escargot une délicieuse délicatesse de France...,https://s3-media3.fl.yelpcdn.com/bphoto/mVwgxg...,https://s3-media3.fl.yelpcdn.com/bphoto/XA5QGI...,https://s3-media2.fl.yelpcdn.com/bphoto/azxglN...
3,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,FR: Ca vaut absolument la peine! A savoir: il...,https://s3-media2.fl.yelpcdn.com/bphoto/wdIhzK...,https://s3-media2.fl.yelpcdn.com/bphoto/DDass9...,https://s3-media3.fl.yelpcdn.com/bphoto/QMNELS...
4,KggnM_Z4wOa_JExunaaWHg,Le Temps des Cerises,"Un gros ""Mouais""\nAttiré par les bons comment...",https://s3-media1.fl.yelpcdn.com/bphoto/g3Aabb...,https://s3-media4.fl.yelpcdn.com/bphoto/Fd0Ozo...,https://s3-media2.fl.yelpcdn.com/bphoto/Eo8fY7...


In [6]:

df_rating.head()

,id,name,text_review,rating
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,J'y suis allé pour acheter un foie gras cru à ...,5
1,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Une adresse immanquable. \nLe meilleur magret ...,5
2,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,"Très bon, le prix abordable et le restaurant e...",4
3,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Les plats étaient très bons et suffisamment co...,5
4,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Le vin était bon et à un prix pas déconnant do...,2


In [7]:
df_yelp.to_csv("data/P6_01_dataset_yelp.csv", sep=',', index=False)
df_rating.to_csv("data/P6_01_dataset_rating.csv", sep=',', index=False)